# Runge-Kutta time schemes convergence 

In [ ]:
from trustutils import run

run.introduction("Elie Saikali and Yannick Gorsse.","29/04/2022")

### Description

We consider a classical 2D VDF conduction problem and we look to the order of convergence of 8 implemented Runge-Kutta schemes (low storage, rational and classical methods : RK2, RK2_classique, RRK2, RK3, RK3_classique, RK4, RK4_classique, RK4_classique_3_8). For each times scheme, we consider 5 calculations where the time step is fixed to 1, 0.1, 0.01, 0.001 and 0.0001 seconds respectively.


In [ ]:
run.TRUST_parameters("1.8.5")

In [ ]:
from trustutils import run 

for i in ["RK2", "RK2_Classique", "RRK2", "RK3", "RK3_Classique", "RK4", "RK4_Classique", "RK4_Classique_3_8"]:
    for j in ["1","1e_1", "1e_2", "1e_3", "1e_4"]:
        run.addCase(f"{i}/{j}","jdd.data",nbProcs=1)

run.printCases()
run.runCases()

In [ ]:
run.executeScript("get_erreur")

## Convergence

It is perfect !!

In [ ]:
from trustutils import plot
import numpy as np

dt = [1,1.e-1,1.e-2,1.e-3,1.e-4]
a = plot.Graph("Convergence")
RK2 = plot.loadText("erreur_RK2")
RK2C = plot.loadText("erreur_RK2_Classique")
RRK2 = plot.loadText("erreur_RRK2")
RK3 = plot.loadText("erreur_RK3")
RK3C = plot.loadText("erreur_RK3_Classique")
RK4 = plot.loadText("erreur_RK4")
RK4C = plot.loadText("erreur_RK4_Classique")
RK4C38 = plot.loadText("erreur_RK4_Classique_3_8")

a.add(dt,RK2,label="RK2",marker='-|')
a.add(dt,RK2C,label="RK2_Classique",marker='-x')
a.add(dt,RRK2,label="RRK2",marker='-*')
a.add(dt,RK3,label="RK3",marker='-|')
a.add(dt,RK3C,label="RK3_Classique",marker='-x')
a.add(dt,RK4,label="RK4",marker='-|')
a.add(dt,RK4C,label="RK4_Classique",marker='-x')
a.add(dt,RK4C38,label="RK4_Classique_3_8",marker='-*')

x = np.linspace(0.0001,1,100)
y = x*x*0.01
a.add(x,y,label="ordre 2")
y = x*x*x*0.005
a.add(x,y,label="ordre 3")

y = x*x*x*x*0.001
a.add(x,y,label="ordre 4")

a.label("dt","erreur")
a.scale(xscale='log',yscale='log')

## Computer performance

In [ ]:
from trustutils import run

run.tablePerf()